In [161]:
import keras
import math
import pandas as pd
import numpy as np
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
from torch.utils.data import Dataset, DataLoader
import pickle
pd.set_option('display.max_rows', 500)
import os
import tensorflow as tf
import torch
import torch.nn as nn
from math import sqrt
# import rmse from sklearn
from sklearn.metrics import mean_squared_error


# define random seeds for Neural Networks
torch.manual_seed(0)
np.random.seed(0)
tf.random.set_seed(0)
# ignore warnings jupyter notebook
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# OWRI FRAMEWORK

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [162]:
def merge_trejectory_data(results, trajectory, direction):
    data = pd.DataFrame()
    for intersection_name in results[trajectory][direction]['raw']:
        intersection = results[trajectory][direction]['raw'][intersection_name]
        intersection = intersection.rename(columns={"cars": intersection_name})
        intersection = intersection.set_index(pd.DatetimeIndex(intersection['timestamp']))
        intersection = intersection.drop(columns=['timestamp'])
        data = pd.merge(data, intersection, left_index=True, right_index=True, how='outer')
    data.dropna(inplace=True)
    return data

In [163]:
def merge_trejectory_data_metr(results):
    data = pd.DataFrame()
    for intersection_name in results['raw']:
        intersection = results['raw'][intersection_name]
        intersection = intersection.rename(columns={"cars": intersection_name})
        intersection = intersection.set_index(pd.DatetimeIndex(intersection['timestamp']))
        intersection = intersection.drop(columns=['timestamp'])
        data = pd.merge(data, intersection, left_index=True, right_index=True, how='outer')
    data.dropna(inplace=True)
    return data

In [164]:
def preprocess_df(df,n_obs, n_features, sequence_length, train_portion = 0.8):
    #do scaling:
    scaler = StandardScaler()
    df_train = df[:math.ceil(len(df)*train_portion)].values
    df_test = df[math.ceil(len(df)*(train_portion)):].values
    train_X, train_y = df_train[:, :n_obs], df_train[:, -n_features]
    test_X, test_y = df_test[:, :n_obs], df_test[:, -n_features]
    scl = scaler.fit(train_X) # fit only on training data
    train_X = scl.transform(train_X)
    test_X = scl.transform(test_X)
    train_X = train_X.reshape((train_X.shape[0], sequence_length, n_features))
    test_X = test_X.reshape((test_X.shape[0], sequence_length, n_features))
    return train_X, train_y, test_X, test_y, scl

In [165]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [166]:
class LSTM_uni(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, layer_dim=1, dropout_prob = 0.2, device = 'cpu'):
        super(LSTM_uni, self).__init__()
        self.hidden_dim = hidden_dim # number of hidden units in hidden state
        self.layer_dim = layer_dim # number of stacked lstm layers
        self.device = device
        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob)
        self.fc = nn.Linear(hidden_dim, output_dim) # fully connected layer

    def forward(self, x, future=False):
        # input x is expected to be of shape (batch_dim, seq_dim, feature_dim)
        # hidden and cell states are expected along with input x in LSTMs = (h_0, c_0)
        # Initialize hidden state with zeros (layer_dim, batch_size, hidden_dim)
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim, device=self.device).requires_grad_()
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim, device=self.device).requires_grad_()

        # LSTM output is Outputs: output, (h_n, c_n)
        # output is of shape (batch_dim, seq_dim, hidden_dim), h_n and c_n are of shape (layer_dim, batch_dim, hidden_dim)
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = out[:, -1, :] # only take the last output of the sequence
        out = self.fc(out) # fully connected layer

        return out

In [167]:
def train_model(model, train_X,train_y, loss_fn, optimiser, device, batch_size, epochs=100):
    history = {}
    history['train_loss'] = []

    train_X_loader = DataLoader(train_X, batch_size=batch_size, shuffle=False)
    train_y_loader = DataLoader(train_y, batch_size=batch_size, shuffle=False)

    for epoch in range(epochs):
        history[epoch] = []
        running_loss = 0.0
        for bx, data in enumerate(zip(train_X_loader,train_y_loader)):
            X = data[0].to(device)
            y = data[1].to(device)
            bt = model(X)
            loss = loss_fn(bt.reshape(-1), y.reshape(-1)) # calculate loss for input and recreated output
            history[epoch].append(loss.item())
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()
            running_loss += loss.item()
        epoch_loss = running_loss/train_X.shape[0]
        history['train_loss'].append(epoch_loss)
    
    return history
    

In [168]:
def model_evaluation( model, test_X, device):
    test_X_loader = DataLoader(test_X, batch_size=128, shuffle=False)
    model = model.eval()
    preds = []
    with torch.no_grad():
        for bx, data in enumerate(test_X_loader):
            X = data.to(device)
            bt = model(X)
            preds.append(bt.cpu().numpy())
    preds = np.vstack(preds)
    preds = preds.reshape(-1)
    return preds

#### ---------------------------- Hauge data processing ---------------------------- 

In [201]:
# results save path
outlier_model_name = ''
data_name = 'hauge'
base_result_path = f'../results/{data_name}/LSTM'
# base_result_path = f'drive/MyDrive/OWRI/results/{data_name}/LSTM' # for google colab
exp_name = f'univariate_{outlier_model_name}_outlier_weighted.pkl'
results_save_path = os.path.join(base_result_path, exp_name)

In [202]:
with open(f'../data/{data_name}/processed/featured_fpds_raw.pickle', 'rb') as f:
    results = pickle.load(f)

# with open(f'drive/MyDrive/OWRI/data/{data_name}/processed/featured_fpds_raw.pickle', 'rb') as f:
#     results = pickle.load(f)  # for google colab

In [203]:
# load data of correlated results from pickle file
with open(f'../results/{data_name}/outlier_scores/{outlier_model_name}/correlated_results.pickle', 'rb') as f:
    correlated_results = pickle.load(f)

# with open(f'drive/MyDrive/OWRI/results/{data_name}/outlier_scores/{outlier_model_name}/correlated_results.pickle', 'rb') as f:
#     correlated_results = pickle.load(f)   # for google colab

In [204]:
thresholds = [0,0.05,0.1,0.25,0.5, 0.75, 1] # thresholds for the percentage of correlated columns to keep
# get target intersections for each trajectory and direction
target_intersections={"T1":{"North":"K504", "South":"K561"},
                      "T2":{"North":"K703", "South":"K206"}}
# declare variables
epoch = 100
batch_size = 64
learning_rate = 0.1
hidden_size = 32
num_layers = 1
dropout = 0.2
sequence_length = 12
output_pred = 1 # number of time steps to predict
train_portion = 0.8 # percentage of data to use for training
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
# device = 'cuda' if torch.cuda.is_available() else 'cpu' # for google colab


In [205]:
errors={}
dfs={}
intersection_arrays = []
for trajectory in results.keys():
    errors[trajectory]={}
    print("\n \n Starting trajectory: {}".format(trajectory))
    for direction in results[trajectory]:
        key = trajectory+"_"+direction
        target = target_intersections[trajectory][direction]
        errors[trajectory][direction]={}
        print("Starting direction: {}".format(direction))
        for threshold in thresholds:
            errors[trajectory][direction][threshold]={}
            print("Starting threshold: {}".format(threshold))
            # ------------------------------------ data processing ---------------------------------------- #
            data = merge_trejectory_data(results, trajectory, direction)# get raw data of the current trajectory and direction
            ae_score = correlated_results[key] # AE scores of the current trajectory and direction
            number_of_cols = math.ceil(len(ae_score.columns)*threshold) # number of outlier weighted intersections
            if number_of_cols==0: # if threshold is 0, then use the target intersection only
                number_of_cols=1
            top_corr_df = ae_score.corr()[target].sort_values(ascending=False)[:number_of_cols] # get the top correlated intersections
            isct_inc = top_corr_df.index.tolist()
            df = data[isct_inc].copy(deep=True)
            df = df[ [target] + [ col for col in df.columns if col != target ] ]  #move target var to front of DF
            df = df.mul(top_corr_df, axis=1)
            df = df.astype('float32')
            n_features = len(isct_inc) # number of features (correlated intersections)
            n_obs = sequence_length * n_features # number of columns in the input
            reframed = series_to_supervised(df, sequence_length, output_pred)
            train_X, train_y, test_X, test_y, scl = preprocess_df(reframed, n_obs, n_features, sequence_length)

#             # # ------------------------------------ modelling ---------------------------------------------- #
            # define model, loss function and optimizer
            model = LSTM_uni(input_dim = n_features, hidden_dim = hidden_size, output_dim = output_pred, layer_dim = num_layers, dropout_prob= dropout, device = device)
            model = model.to(device)
            loss_fn = torch.nn.MSELoss()
            optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
            start = time.time()
            history = train_model(model, train_X,train_y, loss_fn, optimiser, device, batch_size, epochs=epoch)
            end = time.time()
            print("Training time: {}".format(end-start))


            # ------------------------------------ evaluation ---------------------------------------------- #
            yhat = model_evaluation( model, test_X , device)
            errors[trajectory][direction][threshold]['RMSE'] = sqrt(mean_squared_error(yhat,test_y))
            errors[trajectory][direction][threshold]['MAE'] = mean_absolute_error(yhat,test_y)
            errors[trajectory][direction][threshold]['R2'] = r2_score(test_y, yhat)
            errors[trajectory][direction][threshold]['history'] = history
            errors[trajectory][direction][threshold]['df'] = pd.DataFrame({"Real":test_y,"Predicted":yhat})
            errors[trajectory][direction][threshold]['train_time'] = end-start
            print(f"RMSE: {errors[trajectory][direction][threshold]['RMSE']}, MAE: {errors[trajectory][direction][threshold]['MAE']}, R2: {errors[trajectory][direction][threshold]['R2']}")


# save errors in save path as pickle file
with open(results_save_path, 'wb') as handle:
    pickle.dump(errors, handle)


 
 Starting trajectory: T1
Starting direction: North
Starting threshold: 0
Training time: 2202.3177268505096
RMSE: 10.285794537851821, MAE: 7.137937068939209, R2: 0.8702443437375684
Starting threshold: 0.05
Training time: 2091.1715800762177
RMSE: 10.28744922871072, MAE: 7.124786376953125, R2: 0.870202588379974
Starting threshold: 0.1


KeyboardInterrupt: 

#### ---------------------------- METR-LA data processing ---------------------------- 

In [190]:
# results save path
outlier_model_name = 'DAE'
data_name = 'METR-LA'
base_result_path = f'../results/{data_name}/LSTM'
exp_name = f'univariate_{outlier_model_name}_outlier_weighted.pkl'
results_save_path = os.path.join(base_result_path, exp_name)

In [ ]:
with open(f'../data/{data_name}/processed/featured_fpds_raw.pickle', 'rb') as f:
    results = pickle.load(f)

# with open(f'drive/MyDrive/OWRI/data/{data_name}/processed/featured_fpds_raw.pickle', 'rb') as f:
#     results = pickle.load(f)  # for google colab

In [ ]:
# load data of correlated results from pickle file
with open(f'../results/{data_name}/outlier_scores/{outlier_model_name}/correlated_results.pickle', 'rb') as f:
    correlated_results = pickle.load(f)

# with open(f'drive/MyDrive/OWRI/results/{data_name}/outlier_scores/{outlier_model_name}/correlated_results.pickle', 'rb') as f:
#     correlated_results = pickle.load(f)   # for google colab

In [193]:
thresholds = [0,0.05,0.1,0.25,0.5, 0.75, 1] # thresholds for the percentage of correlated columns to keep
target_intersections = list(np.random.choice(list(results['raw'].keys()), 5, replace=False)) # select 10 intersections randomly from the 207 intersections
# declare variables
epoch = 100
batch_size = 64
learning_rate = 0.1
hidden_size = 32
num_layers = 1
dropout = 0.2
sequence_length = 12
output_pred = 1 # number of time steps to predict
train_portion = 0.8 # percentage of data to use for training
# device = 'mps' if torch.backends.mps.is_available() else 'cpu'
device = 'cuda' if torch.cuda.is_available() else 'cpu' # for google colab

In [131]:
# ------------------------------------ for metr-la dataset ---------------------------------------- #
errors={}
dfs={}
intersection_arrays = []
for target in target_intersections:
    errors[target]={}
    print("\n \n Starting target: {}".format(target))
    for threshold in thresholds:
        errors[target][threshold]={}
        print("Starting threshold: {}".format(threshold))

        # ------------------------------------ data processing ---------------------------------------- #
        data = merge_trejectory_data_metr(results)# get raw data of the current trajectory and direction
        ae_score = correlated_results['df'] # AE scores of the current trajectory and direction
        number_of_cols = math.ceil(len(ae_score.columns)*threshold) # number of outlier weighted intersections
        if number_of_cols==0: # if threshold is 0, then use the target intersection only
            number_of_cols=1
        top_corr_df = ae_score.corr()[target].sort_values(ascending=False)[:number_of_cols] # get the top correlated intersections
        isct_inc = top_corr_df.index.tolist()
        df = data[isct_inc].copy(deep=True)
        df = df.mul(top_corr_df, axis=1)
        df = df.astype('float32')
        n_features = len(isct_inc) # number of features (correlated intersections)
        n_obs = sequence_length * n_features # number of columns in the input
        reframed = series_to_supervised(df, sequence_length, output_pred)
        train_X, train_y, test_X, test_y, scl = preprocess_df(reframed, n_obs, n_features, sequence_length,train_portion)

        # ------------------------------------ modelling ---------------------------------------------- #
        # define model, loss function and optimizer
        model = LSTM_uni(input_dim = n_features, hidden_dim = hidden_size, output_dim = output_pred, layer_dim = num_layers, dropout_prob= dropout, device = device)
        model = model.to(device)
        loss_fn = torch.nn.MSELoss()
        optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
        start = time.time()
        history = train_model(model, train_X,train_y, loss_fn, optimiser, device, batch_size, epochs=epoch)
        end = time.time()
        print("Training time: {}".format(end-start))

        # ------------------------------------ evaluation ---------------------------------------------- #
        yhat = model_evaluation( model, test_X , device)
        errors[target][threshold]['RMSE'] = sqrt(mean_squared_error(yhat,test_y))
        errors[target][threshold]['MAE'] = mean_absolute_error(yhat,test_y)
        errors[target][threshold]['R2'] = r2_score(test_y, yhat)
        errors[target][threshold]['history'] = history
        errors[target][threshold]['df'] = pd.DataFrame({"Real":test_y,"Predicted":yhat})
        errors[target][threshold]['train_time'] = end-start
        print(f"RMSE: {errors[target][threshold]['RMSE']}, MAE: {errors[target][threshold]['MAE']}, R2: {errors[target][threshold]['R2']}")


# save errors in save path as pickle file
with open(results_save_path, 'wb') as handle:
    pickle.dump(errors, handle)


 
 Starting target: 762329
Starting threshold: 0


KeyboardInterrupt: 

# -------------------------- PREVIOUS LOF MODEL ---------------------------- 

In [213]:
# results save path
outlier_model_name = 'LOF'
data_name = 'hauge'
base_result_path = f'../results/{data_name}/LSTM'
exp_name = f'univariate_{outlier_model_name}_outlier_weighted.pkl'
results_save_path = os.path.join(base_result_path, exp_name)

In [214]:
with open(f'../data/{data_name}/processed/featured_fpds_raw.pickle', 'rb') as f:
    results = pickle.load(f)

# with open(f'drive/MyDrive/OWRI/data/{data_name}/processed/featured_fpds_raw.pickle', 'rb') as f:
#     results = pickle.load(f)  # for google colab

In [215]:
# load data of correlated results from pickle file
with open(f'../results/{data_name}/outlier_scores/{outlier_model_name}/correlated_results.pickle', 'rb') as f:
    correlated_results = pickle.load(f)

# with open(f'drive/MyDrive/OWRI/results/{data_name}/outlier_scores/{outlier_model_name}/correlated_results.pickle', 'rb') as f:
#     correlated_results = pickle.load(f)   # for google colab

In [216]:
thresholds = [0,0.05,0.1,0.25,0.5, 1] # thresholds for the percentage of correlated columns to keep
# get target intersections for each trajectory and direction
target_intersections={"T1":{"North":"K504", "South":"K561"},
                      "T2":{"North":"K703", "South":"K206"}}
# declare variables
epoch = 100
batch_size = 64
learning_rate = 0.1
hidden_size = 32
num_layers = 1
dropout = 0.2
sequence_length = 12
output_pred = 1 # number of time steps to predict
train_portion = 0.8 # percentage of data to use for training
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
# device = 'cuda' if torch.cuda.is_available() else 'cpu' # for google colab


In [217]:
errors={}
dfs={}
intersection_arrays = []
for trajectory in results.keys():
    errors[trajectory]={}
    print("\n \n Starting trajectory: {}".format(trajectory))
    for direction in results[trajectory]:
        # key = trajectory+"_"+direction
        target = target_intersections[trajectory][direction]
        errors[trajectory][direction]={}
        print("Starting direction: {}".format(direction))
        for threshold in thresholds:
            errors[trajectory][direction][threshold]={}
            print("Starting threshold: {}".format(threshold))
            # ------------------------------------ data processing ---------------------------------------- #
            data = merge_trejectory_data(results, trajectory, direction)# get raw data of the current trajectory and direction
            ae_score = correlated_results[trajectory][direction] # AE scores of the current trajectory and direction
            number_of_cols = math.ceil(len(ae_score.columns)*threshold) # number of outlier weighted intersections
            if number_of_cols==0: # if threshold is 0, then use the target intersection only
                number_of_cols=1
            top_corr_df = ae_score.corr()[target].sort_values(ascending=False)[:number_of_cols] # get the top correlated intersections
            isct_inc = top_corr_df.index.tolist()
            df = data[isct_inc].copy(deep=True)
            df = df[ [target] + [ col for col in df.columns if col != target ] ]  #move target var to front of DF
            # df = df.mul(top_corr_df, axis=1)
            df = df.astype('float32')
            n_features = len(isct_inc) # number of features (correlated intersections)
            n_obs = sequence_length * n_features # number of columns in the input
            reframed = series_to_supervised(df, sequence_length, output_pred)
            train_X, train_y, test_X, test_y, scl = preprocess_df(reframed, n_obs, n_features, sequence_length)

#             # # ------------------------------------ modelling ---------------------------------------------- #
            # define model, loss function and optimizer
            model = LSTM_uni(input_dim = n_features, hidden_dim = hidden_size, output_dim = output_pred, layer_dim = num_layers, dropout_prob= dropout, device = device)
            model = model.to(device)
            loss_fn = torch.nn.MSELoss()
            optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
            start = time.time()
            history = train_model(model, train_X,train_y, loss_fn, optimiser, device, batch_size, epochs=epoch)
            end = time.time()
            print("Training time: {}".format(end-start))


            # ------------------------------------ evaluation ---------------------------------------------- #
            yhat = model_evaluation( model, test_X , device)
            errors[trajectory][direction][threshold]['RMSE'] = sqrt(mean_squared_error(yhat,test_y))
            errors[trajectory][direction][threshold]['MAE'] = mean_absolute_error(yhat,test_y)
            errors[trajectory][direction][threshold]['R2'] = r2_score(test_y, yhat)
            errors[trajectory][direction][threshold]['history'] = history
            errors[trajectory][direction][threshold]['df'] = pd.DataFrame({"Real":test_y,"Predicted":yhat})
            errors[trajectory][direction][threshold]['train_time'] = end-start
            print(f"RMSE: {errors[trajectory][direction][threshold]['RMSE']}, MAE: {errors[trajectory][direction][threshold]['MAE']}, R2: {errors[trajectory][direction][threshold]['R2']}")


# save errors in save path as pickle file
with open(results_save_path, 'wb') as handle:
    pickle.dump(errors, handle)


 
 Starting trajectory: T1
Starting direction: North
Starting threshold: 0
Training time: 2041.0620119571686
RMSE: 10.450731490966987, MAE: 7.246530055999756, R2: 0.8660496032517396
Starting threshold: 0.05


array([84.686295 , 81.41526  , 82.89845  , ...,  1.3487635,  1.3487635,
        1.3487635], dtype=float32)